# 한국어 금융보안 데이터셋 번역 작업

## 작업 개요
이 노트북은 영어/중국어로 작성된 금융보안 관련 QA 데이터셋을 한국어로 번역하는 작업을 수행합니다.

## 대상 파일들
1. **CyberMetric/mcqa_org.csv** - 영어 전용 MCQA 데이터
2. **SecBench/mcqa_org.csv** - 영어/중국어 혼용 MCQA 데이터  
3. **SecBench/qa_org.csv** - 영어/중국어 혼용 QA 데이터

## 작업 흐름
1. 각 `_org.csv` 파일을 읽어서 데이터 구조 및 언어 분포 확인
2. **언어 자동 감지**: SecBench 데이터의 영어/중국어 혼용 상황 처리
3. 로컬 LLM을 사용하여 한국어로 번역
4. 번역된 결과를 원래 폴더에 한국어 버전으로 저장
5. 번역 품질 검증

## 주요 개선사항
- **언어 자동 감지 기능**: SecBench 데이터의 영어/중국어 혼용 문제 해결
- **개별 항목별 언어 판단**: 질문과 선택지마다 독립적으로 언어 감지
- **이미 번역된 내용 건너뛰기**: 한국어 텍스트는 재번역하지 않음

In [ ]:
# 필요한 라이브러리 설치 및 설정

%pip install -q transformers==4.55.0  # LLM 요구사항: >=4.46.0
%pip install -q safetensors==0.4.3    # torch 2.1.0 호환성
%pip install -q bitsandbytes==0.43.2 accelerate==1.9.0  # 양자화 지원
%pip install -q torch torchaudio

print("라이브러리 설치 완료!")

import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import json
import re
from typing import List, Dict, Tuple
import time

In [ ]:
# LLM 모델 로드 및 번역 함수 정의

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
import unicodedata

def load_translation_model(model_name="microsoft/DialoGPT-medium"):
    """번역용 LLM 모델을 로드합니다."""
    try:
        # 양자화 설정 (메모리 절약)
        quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.float16,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4"
        )
        
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            quantization_config=quantization_config,
            device_map="auto",
            torch_dtype=torch.float16
        )
        
        # pad_token 설정
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token
            
        print(f"모델 '{model_name}' 로드 완료!")
        return tokenizer, model
        
    except Exception as e:
        print(f"모델 로드 실패: {e}")
        print("대안: OpenAI API 또는 다른 번역 서비스를 사용하세요.")
        return None, None

def detect_language(text: str) -> str:
    """텍스트의 언어를 자동 감지합니다."""
    
    # 한국어는 이미 번역 완료된 것으로 간주
    korean_chars = sum(1 for char in text if '\uac00' <= char <= '\ud7af')
    if korean_chars > len(text) * 0.1:  # 10% 이상이 한글이면 한국어
        return "ko"
    
    # 중국어 문자 감지 (간체/번체 포함)
    chinese_chars = sum(1 for char in text if '\u4e00' <= char <= '\u9fff')
    
    # 영어 문자 감지 (알파벳)
    english_chars = sum(1 for char in text if char.isascii() and char.isalpha())
    
    # 전체 텍스트 길이
    total_chars = len(text.replace(' ', '').replace('\n', ''))
    
    if total_chars == 0:
        return "unknown"
    
    # 중국어 비율이 높으면 중국어
    if chinese_chars > total_chars * 0.3:
        return "zh"
    
    # 영어 비율이 높으면 영어
    if english_chars > total_chars * 0.5:
        return "en"
    
    # 혼용되어 있는 경우 더 많은 쪽으로 판단
    if chinese_chars > english_chars:
        return "zh"
    elif english_chars > chinese_chars:
        return "en"
    else:
        return "unknown"

def translate_text(text: str, tokenizer, model, source_lang: str = "auto", target_lang: str = "ko") -> str:
    """텍스트를 한국어로 번역합니다. source_lang이 'auto'면 자동 감지합니다."""
    
    # 언어 자동 감지
    if source_lang == "auto":
        detected_lang = detect_language(text)
        if detected_lang == "ko":
            return text  # 이미 한국어면 번역하지 않음
        elif detected_lang == "unknown":
            # 알 수 없는 경우 영어로 가정
            detected_lang = "en"
        source_lang = detected_lang
    
    # 번역 프롬프트 구성
    if source_lang == "en":
        prompt = f"Translate the following English text to Korean. Maintain the technical terminology and formatting:\n\nEnglish: {text}\nKorean:"
    elif source_lang == "zh":
        prompt = f"Translate the following Chinese text to Korean. Maintain the technical terminology and formatting:\n\nChinese: {text}\nKorean:"
    else:
        prompt = f"Translate to Korean: {text}\nKorean:"
    
    try:
        # 토큰화
        inputs = tokenizer.encode(prompt, return_tensors="pt", truncation=True, max_length=512)
        
        # 생성
        with torch.no_grad():
            outputs = model.generate(
                inputs,
                max_new_tokens=200,
                num_return_sequences=1,
                temperature=0.7,
                do_sample=True,
                pad_token_id=tokenizer.eos_token_id
            )
        
        # 디코딩
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        # 번역 결과 추출
        if "Korean:" in generated_text:
            translated = generated_text.split("Korean:")[-1].strip()
        else:
            translated = generated_text[len(prompt):].strip()
            
        return translated if translated else text  # 번역 실패시 원문 반환
        
    except Exception as e:
        print(f"번역 오류: {e}")
        return text  # 오류시 원문 반환

# 모델 로드 (실제 사용시 적절한 모델명으로 변경)
print("LLM 모델 로딩 중...")
# tokenizer, model = load_translation_model("beomi/gemma-ko-7b")  # 예시 모델명
print("실제 사용시 적절한 모델명을 설정하세요.")

In [ ]:
# 데이터 로드 및 구조 확인

# 1. CyberMetric MCQA (영어 전용)
print("=== CyberMetric MCQA 데이터 ===")
cyber_mcqa = pd.read_csv('../data/CyberMetric/mcqa_org.csv')
print(f"데이터 크기: {cyber_mcqa.shape}")
print(f"컬럼: {cyber_mcqa.columns.tolist()}")
print("\n샘플 데이터:")
print(cyber_mcqa.head(2))

print("\n" + "="*50 + "\n")

# 2. SecBench MCQA (영어/중국어 혼용)
print("=== SecBench MCQA 데이터 ===")
sec_mcqa = pd.read_csv('../data/SecBench/mcqa_org.csv')
print(f"데이터 크기: {sec_mcqa.shape}")
print(f"컬럼: {sec_mcqa.columns.tolist()}")
print("\n샘플 데이터:")
print(sec_mcqa.head(2))

# 언어 분포 확인
print("\n언어 분포 분석:")
languages = []
for idx in range(min(100, len(sec_mcqa))):  # 처음 100개만 샘플링
    question = sec_mcqa.iloc[idx]['Question']
    lang = detect_language(question.split('\n')[0])  # 첫 번째 줄만 검사
    languages.append(lang)

from collections import Counter
lang_count = Counter(languages)
print(f"샘플 100개 중 언어 분포: {dict(lang_count)}")

print("\n" + "="*50 + "\n")

# 3. SecBench QA (영어/중국어 혼용)
print("=== SecBench QA 데이터 ===")
sec_qa = pd.read_csv('../data/SecBench/qa_org.csv')
print(f"데이터 크기: {sec_qa.shape}")
print(f"컬럼: {sec_qa.columns.tolist()}")
print("\n샘플 데이터:")
print(sec_qa.head(2))

# 언어 분포 확인
print("\n언어 분포 분석:")
qa_languages = []
for idx in range(min(100, len(sec_qa))):  # 처음 100개만 샘플링
    question = sec_qa.iloc[idx]['Question']
    lang = detect_language(question)
    qa_languages.append(lang)

qa_lang_count = Counter(qa_languages)
print(f"QA 샘플 100개 중 언어 분포: {dict(qa_lang_count)}")

In [ ]:
# MCQA 데이터 번역 함수

def parse_mcqa_question(question_text: str) -> Dict:
    """MCQA 질문을 파싱하여 문제와 선택지를 분리합니다."""
    lines = question_text.strip().split('\n')
    
    # 첫 번째 줄은 문제 
    question = lines[0]
    
    # 나머지는 선택지
    choices = []
    for line in lines[1:]:
        line = line.strip()
        if line and (line.startswith(('1.', '2.', '3.', '4.', '5.', '6.', '7.'))):
            choices.append(line)
    
    return {
        'question': question,
        'choices': choices
    }

def translate_mcqa_entry(question_text: str, answer: str, tokenizer, model, source_lang: str = "auto") -> Tuple[str, str]:
    """MCQA 엔트리를 번역합니다. 언어 자동 감지 지원."""
    
    # 질문 파싱
    parsed = parse_mcqa_question(question_text)
    
    # 질문 번역 (언어 자동 감지)
    if source_lang == "auto":
        detected_lang = detect_language(parsed['question'])
        print(f"질문 언어 감지: {detected_lang}")
    else:
        detected_lang = source_lang
    
    translated_question = translate_text(parsed['question'], tokenizer, model, detected_lang)
    
    # 각 선택지 번역
    translated_choices = []
    for choice in parsed['choices']:
        # 선택지 번호와 내용 분리
        if '. ' in choice:
            num_part, content = choice.split('. ', 1)
            # 각 선택지마다 언어 감지 (혼용 가능성)
            choice_lang = detect_language(content) if source_lang == "auto" else detected_lang
            translated_content = translate_text(content, tokenizer, model, choice_lang)
            translated_choices.append(f"{num_part}. {translated_content}")
        else:
            choice_lang = detect_language(choice) if source_lang == "auto" else detected_lang
            translated_choices.append(translate_text(choice, tokenizer, model, choice_lang))
    
    # 번역된 질문과 선택지 결합
    translated_full = translated_question + '\n' + '\n'.join(translated_choices)
    
    # 답변은 그대로 유지 (번호나 간단한 형태)
    translated_answer = answer
    
    return translated_full, translated_answer

def translate_qa_entry(question: str, answer: str, tokenizer, model, source_lang: str = "auto") -> Tuple[str, str]:
    """QA 엔트리를 번역합니다. 언어 자동 감지 지원."""
    
    # 질문과 답변 각각 언어 감지
    if source_lang == "auto":
        q_lang = detect_language(question)
        a_lang = detect_language(answer)
        print(f"질문 언어: {q_lang}, 답변 언어: {a_lang}")
    else:
        q_lang = a_lang = source_lang
    
    translated_question = translate_text(question, tokenizer, model, q_lang)
    translated_answer = translate_text(answer, tokenizer, model, a_lang)
    
    return translated_question, translated_answer

print("번역 함수 정의 완료!")

In [ ]:
# 1. CyberMetric MCQA 번역 (영어 → 한국어)

def translate_cybermetric_mcqa(df: pd.DataFrame, tokenizer, model, batch_size: int = 5):
    """CyberMetric MCQA 데이터를 번역합니다."""
    
    translated_data = []
    
    print(f"CyberMetric MCQA 총 {len(df)}개 항목 번역 시작...")
    
    for idx, row in tqdm(df.iterrows(), total=len(df)):
        try:
            question = row['Question']
            answer = row['Answer']
            
            # 번역 수행 (CyberMetric은 영어만)
            translated_question, translated_answer = translate_mcqa_entry(
                question, answer, tokenizer, model, source_lang="en"
            )
            
            translated_data.append({
                'Question': translated_question,
                'Answer': translated_answer
            })
            
            # 배치마다 잠시 대기 (메모리 관리)
            if (idx + 1) % batch_size == 0:
                time.sleep(1)
                
        except Exception as e:
            print(f"번역 오류 (행 {idx}): {e}")
            # 오류시 원본 데이터 사용
            translated_data.append({
                'Question': row['Question'], 
                'Answer': row['Answer']
            })
    
    return pd.DataFrame(translated_data)

# 실제 번역 실행 (모델이 로드된 경우)
if 'tokenizer' in locals() and 'model' in locals() and tokenizer is not None:
    cyber_mcqa_ko = translate_cybermetric_mcqa(cyber_mcqa.head(10), tokenizer, model)  # 테스트용 10개만
    print("CyberMetric MCQA 번역 완료!")
    print(cyber_mcqa_ko.head(2))
else:
    print("모델이 로드되지 않았습니다. 실제 실행시 모델을 먼저 로드하세요.")


In [ ]:
# 2. SecBench MCQA 번역 (중국어 → 한국어)

def translate_secbench_mcqa(df: pd.DataFrame, tokenizer, model, batch_size: int = 5):
    """SecBench MCQA 데이터를 번역합니다."""
    
    translated_data = []
    
    print(f"SecBench MCQA 총 {len(df)}개 항목 번역 시작...")
    
    for idx, row in tqdm(df.iterrows(), total=len(df)):
        try:
            question = row['Question']
            answer = row['Answer']
            
            # 번역 수행 (SecBench는 영어/중국어 혼용, 자동 감지)
            translated_question, translated_answer = translate_mcqa_entry(
                question, answer, tokenizer, model, source_lang="auto"
            )
            
            translated_data.append({
                'Question': translated_question,
                'Answer': translated_answer
            })
            
            # 배치마다 잠시 대기
            if (idx + 1) % batch_size == 0:
                time.sleep(1)
                
        except Exception as e:
            print(f"번역 오류 (행 {idx}): {e}")
            translated_data.append({
                'Question': row['Question'], 
                'Answer': row['Answer']
            })
    
    return pd.DataFrame(translated_data)

# 실제 번역 실행
if 'tokenizer' in locals() and 'model' in locals() and tokenizer is not None:
    sec_mcqa_ko = translate_secbench_mcqa(sec_mcqa.head(10), tokenizer, model)  # 테스트용 10개만
    print("SecBench MCQA 번역 완료!")
    print(sec_mcqa_ko.head(2))
else:
    print("모델이 로드되지 않았습니다. 실제 실행시 모델을 먼저 로드하세요.")


In [ ]:
# 3. SecBench QA 번역 (중국어 → 한국어)

def translate_secbench_qa(df: pd.DataFrame, tokenizer, model, batch_size: int = 5):
    """SecBench QA 데이터를 번역합니다."""
    
    translated_data = []
    
    print(f"SecBench QA 총 {len(df)}개 항목 번역 시작...")
    
    for idx, row in tqdm(df.iterrows(), total=len(df)):
        try:
            question = row['Question']
            answer = row['Answer']
            
            # 번역 수행 (SecBench는 영어/중국어 혼용, 자동 감지)
            translated_question, translated_answer = translate_qa_entry(
                question, answer, tokenizer, model, source_lang="auto"
            )
            
            translated_data.append({
                'Question': translated_question,
                'Answer': translated_answer
            })
            
            # 배치마다 잠시 대기
            if (idx + 1) % batch_size == 0:
                time.sleep(1)
                
        except Exception as e:
            print(f"번역 오류 (행 {idx}): {e}")
            translated_data.append({
                'Question': row['Question'], 
                'Answer': row['Answer']
            })
    
    return pd.DataFrame(translated_data)

# 실제 번역 실행
if 'tokenizer' in locals() and 'model' in locals() and tokenizer is not None:
    sec_qa_ko = translate_secbench_qa(sec_qa.head(10), tokenizer, model)  # 테스트용 10개만
    print("SecBench QA 번역 완료!")
    print(sec_qa_ko.head(2))
else:
    print("모델이 로드되지 않았습니다. 실제 실행시 모델을 먼저 로드하세요.")


In [ ]:
# 번역 결과 저장

def save_translated_data():
    """번역된 데이터를 CSV 파일로 저장합니다."""
    
    # 저장할 데이터와 경로 정의
    datasets_to_save = [
        {
            'data': 'cyber_mcqa_ko',
            'path': '../data/CyberMetric/mcqa.csv',
            'description': 'CyberMetric MCQA (영어→한국어)'
        },
        {
            'data': 'sec_mcqa_ko', 
            'path': '../data/SecBench/mcqa.csv',
            'description': 'SecBench MCQA (중국어→한국어)'
        },
        {
            'data': 'sec_qa_ko',
            'path': '../data/SecBench/qa.csv', 
            'description': 'SecBench QA (중국어→한국어)'
        }
    ]
    
    for dataset_info in datasets_to_save:
        data_name = dataset_info['data']
        file_path = dataset_info['path']
        description = dataset_info['description']
        
        try:
            # 변수가 존재하는지 확인
            if data_name in locals() or data_name in globals():
                data = locals().get(data_name) or globals().get(data_name)
                
                if data is not None and len(data) > 0:
                    # CSV로 저장
                    data.to_csv(file_path, index=False, encoding='utf-8')
                    print(f"✅ {description} 저장 완료: {file_path} ({len(data)}개 항목)")
                else:
                    print(f"❌ {description}: 데이터가 비어있습니다.")
            else:
                print(f"❌ {description}: 변수 '{data_name}'를 찾을 수 없습니다.")
                
        except Exception as e:
            print(f"❌ {description} 저장 실패: {e}")

# 실제 번역이 완료된 경우에만 저장
print("번역 결과 저장 함수 준비 완료!")
print("실제 번역 완료 후 save_translated_data() 함수를 호출하세요.")


In [ ]:
# 번역 품질 검증 및 샘플 확인

def validate_translation_quality(original_df: pd.DataFrame, translated_df: pd.DataFrame, 
                                dataset_name: str, sample_size: int = 5):
    """번역 품질을 검증하고 샘플을 확인합니다."""
    
    print(f"\n=== {dataset_name} 번역 품질 검증 ===")
    print(f"원본 데이터 크기: {len(original_df)}")
    print(f"번역 데이터 크기: {len(translated_df)}")
    
    if len(original_df) != len(translated_df):
        print("⚠️ 경고: 원본과 번역본의 데이터 크기가 다릅니다!")
    
    # 샘플 비교
    print(f"\n📝 샘플 {sample_size}개 비교:")
    for i in range(min(sample_size, len(original_df), len(translated_df))):
        print(f"\n--- 샘플 {i+1} ---")
        print(f"원본 질문: {original_df.iloc[i]['Question'][:100]}...")
        print(f"번역 질문: {translated_df.iloc[i]['Question'][:100]}...")
        print(f"원본 답변: {original_df.iloc[i]['Answer']}")
        print(f"번역 답변: {translated_df.iloc[i]['Answer']}")
        print("-" * 50)

# 실제 번역 완료 후 품질 검증 실행
def run_validation_after_translation():
    """번역 완료 후 품질 검증을 실행합니다."""
    
    try:
        if 'cyber_mcqa_ko' in locals() or 'cyber_mcqa_ko' in globals():
            validate_translation_quality(cyber_mcqa, cyber_mcqa_ko, "CyberMetric MCQA")
    except NameError:
        print("CyberMetric MCQA 번역 데이터를 찾을 수 없습니다.")
    
    try:
        if 'sec_mcqa_ko' in locals() or 'sec_mcqa_ko' in globals():
            validate_translation_quality(sec_mcqa, sec_mcqa_ko, "SecBench MCQA")
    except NameError:
        print("SecBench MCQA 번역 데이터를 찾을 수 없습니다.")
    
    try:
        if 'sec_qa_ko' in locals() or 'sec_qa_ko' in globals():
            validate_translation_quality(sec_qa, sec_qa_ko, "SecBench QA")
    except NameError:
        print("SecBench QA 번역 데이터를 찾을 수 없습니다.")

print("번역 품질 검증 함수 준비 완료!")


# 전체 번역 작업 실행 가이드

## 단계별 실행 방법

### 1. 환경 설정
1. 첫 번째 셀부터 "LLM 모델 로드" 셀까지 실행
2. 적절한 번역 모델 선택 및 로드

### 2. 모델 선택 옵션
- **로컬 모델**: `beomi/gemma-ko-7b`, `microsoft/DialoGPT-medium` 등
- **API 기반**: OpenAI GPT, Google Translate API, Papago API
- **권장**: 한국어에 특화된 모델 사용

### 3. 번역 실행
```python
# 모델 로드 (적절한 모델명으로 변경)
tokenizer, model = load_translation_model("your-model-name")

# 전체 데이터 번역 (head(10) 제거)
cyber_mcqa_ko = translate_cybermetric_mcqa(cyber_mcqa, tokenizer, model)
sec_mcqa_ko = translate_secbench_mcqa(sec_mcqa, tokenizer, model)  
sec_qa_ko = translate_secbench_qa(sec_qa, tokenizer, model)

# 결과 저장
save_translated_data()

# 품질 검증
run_validation_after_translation()
```

### 4. 주의사항
- 메모리 부족시 `batch_size`를 줄이세요
- 번역 속도가 느린 경우 더 작은 모델을 사용하세요
- GPU 메모리가 부족한 경우 4bit 양자화를 활용하세요

### 5. 출력 파일
- `../data/CyberMetric/mcqa.csv` - 영어→한국어 MCQA
- `../data/SecBench/mcqa.csv` - 중국어→한국어 MCQA  
- `../data/SecBench/qa.csv` - 중국어→한국어 QA
